# Visual Navigation in Real-World Indoor Environments Using End-to-End Deep Reinforcement Learning
[Jonáš Kulhánek](https://jkulhanek.github.io/), [Erik Derner](http://people.ciirc.cvut.cz/~derneeri/), [Robert Babuška](http://www.robertbabuska.com/)<br>

In this notebook, you can play with the pre-trained models and evaluate them both the synthetic and real-world dataset. Start by running the first cell.

## Contents
- [Simulated environment (DMHouse)](#Simulated-environment-DMHouse)
- [Real-world dataset](#Real-world-dataset)

In [ ]:
# Installs dependencies
!apt-get install libsdl2-dev libosmesa6-dev gettext g++ unzip zip curl gnupg libstdc++6

# Installs the source code and python dependencies
!rm -rf robot-visual-navigation && git clone https://github.com/jkulhanek/robot-visual-navigation.git
!pip install -r robot-visual-navigation/requirements.txt
!pip install matplotlib==3.1.3

import sys; sys.path.append('robot-visual-navigation/python')
import deep_rl
from deep_rl import make_trainer
import deep_rl.configuration as config
from configuration import configuration
import trainer as _

deep_rl.configure(**configuration)

# Simulated environment (DMHouse)
[back to top 🠕](#Visual-Navigation-in-Real-World-Indoor-Environments-Using-End-to-End-Deep-Reinforcement-Learning)

In [ ]:
#
# Download DMHouse models
#

!mkdir -p ~/.cache/robot-visual-navigation/models
!curl -L https://data.ciirc.cvut.cz/public/projects/2021RealWorldNavigation/checkpoints/dmhouse-models.tar.gz | tar -xz -C ~/.cache/robot-visual-navigation/models

In [8]:
#
# Generate and render a single episode
#
import dmhouse
import gym
from trainer import create_wrapped_environment
from deep_rl import make_agent
from evaluate_dmhouse import ExpandDimWrapper
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import animation
rc('animation', html='jshtml')

MODEL_NAME = 'dmhouse'
env = create_wrapped_environment(id='DMHouseCustom-v1', same_map_episodes=1)
agent = make_agent(MODEL_NAME)
env = agent.wrap_env(env)
agent = ExpandDimWrapper(agent)
if hasattr(agent.unwrapped, "set_environment"):
    agent.unwrapped.set_environment(env)

# Run the episode
o = env.reset()
observations = [o[0][:2]]
agent.reset_state()
actions = []
terminated = False
max_env_steps = 1000
reward = 0
while not terminated and max_env_steps > 0:
    action = agent.act(o)
    actions.append(action)
    o, reward, terminated, _ = env.step(action)
    observations.append(o[0][:2])
    max_env_steps -= 1

# Show video
fig = plt.figure(figsize=(8, 4))
p1, p2 = fig.subplots(1, 2)
i1 = p1.imshow(observations[0][0].transpose(1, 2, 0))
p1.set_title('observation', fontsize=16, y=-0.14)
p1.axis('off')
i2 = p2.imshow(observations[0][1].transpose(1, 2, 0))
p2.set_title('goal', fontsize=16, y=-0.14)
p2.axis('off')

def ani(index):
    i1.set_data(observations[index][0].transpose(1, 2, 0))
    i2.set_data(observations[index][1].transpose(1, 2, 0))

animator = animation.FuncAnimation(fig, ani, frames=len(observations), interval=100, blit=False, repeat=True)
plt.close()
animator

# Real-world dataset
[back to top 🠕](#Visual-Navigation-in-Real-World-Indoor-Environments-Using-End-to-End-Deep-Reinforcement-Learning)


In [ ]:
#
# Download real-world dataset and models
# NOTE: This takes a long time!
#
!curl -L https://data.ciirc.cvut.cz/public/projects/2021RealWorldNavigation/checkpoints/turtlebot-models.tar.gz | tar -xz -C ~/.cache/robot-visual-navigation/models
!mkdir -p ~/.cache/robot-visual-navigation/datasets
!curl -L -o ~/.cache/robot-visual-navigation/datasets/turtle_room_compiled.hdf5 https://data.ciirc.cvut.cz/public/projects/2021RealWorldNavigation/datasets/turtle_room_compiled.hdf5

In [32]:
#
# Generate and render a single episode
#
import dmhouse
import gym
from trainer import create_wrapped_environment
from deep_rl import make_agent
from evaluate_dmhouse import ExpandDimWrapper
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import animation
rc('animation', html='jshtml')

MODEL_NAME = 'turtlebot'
env = create_wrapped_environment(id="TurtleLab-v0", has_end_action=True, augment_images=False)
agent = make_agent(MODEL_NAME)
observations = []

class GrabImageWrapper(gym.Wrapper):
    def reset(self, *args, **kwargs):
        out = self.env.reset(*args, **kwargs)
        observations.append(out[:2])
        return out

    def step(self, *args, **kwargs):
        out = self.env.step(*args, **kwargs)
        observations.append(out[0][:2])
        return out

env.env.env.env.env.env = GrabImageWrapper(env.env.env.env.env.env)
env = agent.wrap_env(env)
agent = ExpandDimWrapper(agent)
if hasattr(agent.unwrapped, "set_environment"):
    agent.unwrapped.set_environment(env)

# Run the episode
o = env.reset()
agent.reset_state()
actions = []
terminated = False
max_env_steps = 1000
reward = 0
while not terminated and max_env_steps > 0:
    action = agent.act(o)
    actions.append(action)
    o, reward, terminated, _ = env.step(action)
    max_env_steps -= 1

# Show video
fig = plt.figure(figsize=(8, 4))
p1, p2 = fig.subplots(1, 2)
i1 = p1.imshow(observations[0][0])
p1.set_title('observation', fontsize=16, y=-0.14)
p1.axis('off')
i2 = p2.imshow(observations[0][1])
p2.set_title('goal', fontsize=16, y=-0.14)
p2.axis('off')

def ani(index):
    i1.set_data(observations[index][0])
    i2.set_data(observations[index][1])

animator = animation.FuncAnimation(fig, ani, frames=len(observations), interval=200, blit=False, repeat=True)
plt.close()
animator